In [1]:
using JuMP
using MixedIntegerExperiments
using Polyhedra
using CDDLib
# using Plots
using DrakeVisualizer
using AxisArrayVariables
using AxisArrays
using BenchmarkTools
using StaticArrays
import BoxRobots
using DataFrames
# plotlyjs();

Loading cdd global constants


In [2]:
function bilinear_relaxation_method_eval(method::Symbol, disc_level::Int)
    optparams = MIQPTrajOptParams(verbose = false, nsteps = 10, disc_level = disc_level, bilinearmethod = method)
    contact_point_descriptions = Dict(:foot => ContactPointDescription(2., SimpleVRepresentation([-0.5 -0.5; 0.5 -0.5; -0.5 -1.; 0.5 -1.])))
    robot = BoxRobotWithRotation2D(0., 1., 60., SVector(0, -9.81), contact_point_descriptions)
    environment = [
        contact_region([0.; 0.], [1.; 0.], 0.8, 2 * robot.m * norm(robot.g)); 
        axis_aligned_free_box_region([0.; 0.], [1.; 1.])];

    contactpointstates = Dict(:foot => ContactPointState(SVector(0.5, 0.)))
    initialstate = BoxRobotWithRotation2DState(0., 0.1, SVector(0.5, 0.8), SVector(0.5, 0.), contactpointstates)
    states, inputs, diagnostics = miqp_trajopt(robot, environment, initialstate, optparams);
end

bilinear_relaxation_method_eval (generic function with 1 method)

In [3]:
function bilinear_relaxation_method_eval(scenarios::Dict{Symbol, Vector{Int}})
    methods = Symbol[]
    levels = Int[]
    times = Float64[]
    violations = Float64[]

    for (method, disc_levels) in scenarios
        for disc_level in disc_levels
            println("Evaluating $(string(method)) at discretization level $(disc_level - 1)")
            flush(STDOUT)
            states, inputs, diagnostics = bilinear_relaxation_method_eval(method, disc_level)
            time = diagnostics.solvetime
            violation = constraint_violation_norm(diagnostics)
            push!(methods, method)
            push!(levels, disc_level - 1) # -1, because I want 1 to mean a single McCormick envelope
            push!(times, time)
            push!(violations, violation)
            println("time: $time, violation: $violation")
            println()
        end
    end

    d = DataFrame(method = methods, disclevel = levels, time = times, violation = violations)
end

bilinear_relaxation_method_eval (generic function with 2 methods)

In [4]:
# scenarios = Dict(:HRepConvexHull => [4, 9], :Logarithmic1D => [4, 9], :Logarithmic2D => [2, 3])
# scenarios = Dict(:HRepConvexHull => collect(2 : 11), :Logarithmic1D => collect(2 : 18), :Logarithmic2D => collect(2 : 9))
# bilinear_relaxation_method_eval(Dict(:HRepConvexHull => [2])) # warmup
# d = bilinear_relaxation_method_eval(scenarios)

In [25]:
datadir = "/home/twan/Dropbox/170612 group meeting long/data"
figuredir = "/home/twan/Dropbox/170612 group meeting long/figures"
filepath = joinpath(datadir, "bilinearrelax.csv")
# writetable(filepath, d)
d = readtable(filepath);

In [26]:
using Plots
using StatPlots
using Query

In [34]:
tmax = maximum(d[:time])
violmax = maximum(d[:violation])
sz = 0.75 .* [400; 300]
for method in unique(d[:method])
    x = @from i in d begin
        @where i.method == method
        @select {i.disclevel, i.time, i.violation}
        @collect DataFrame
    end
    xticks = Vector(x[:disclevel])
    timeplot = bar(x[:disclevel], x[:time], xlabel = "discretization level", ylabel = "solve time [s]", 
    lab = "", title = method, ylim = [0., tmax], xticks = xticks, size = sz)
    savefig(timeplot, joinpath(figuredir, "bilinearrelax_$(method)_time.png"))
#     display(timeplot)
    
    violplot = bar(x[:disclevel], x[:violation], xlabel = "discretization level", ylabel = "constraint violation [Nm]",
        lab = "", title = method, color = :red, ylim = [0., violmax], xticks = xticks, size = sz)
    savefig(violplot, joinpath(figuredir, "bilinearrelax_$(method)_viol.png"))
#     display(violplot)
end

In [5]:
show_plots = false;

In [6]:
# if show_plots
#     display(plot_environment(environment))
#     for (i, region) in enumerate(environment)
#         isfree(region) || display(plot_allowable_forces(region, i))
#     end
#     display(plot_kinematic_regions(Dict(sym => limb.kinematic_region for (sym, limb) in robot.ContactPointDescriptions)))
# end

In [7]:
method = :Logarithmic2D
disc_level = 6
states, inputs, diagnostics = bilinear_relaxation_method_eval(method, disc_level);

In [8]:
optparams = MIQPTrajOptParams(verbose = false, nsteps = 10, disc_level = disc_level, bilinearmethod = method)
contact_point_descriptions = Dict(:foot => ContactPointDescription(2., SimpleVRepresentation([-0.5 -0.5; 0.5 -0.5; -0.5 -1.; 0.5 -1.])))
robot = BoxRobotWithRotation2D(0., 1., 60., SVector(0, -9.81), contact_point_descriptions)
environment = [
    contact_region([0.; 0.], [1.; 0.], 0.8, 2 * robot.m * norm(robot.g)); 
    axis_aligned_free_box_region([0.; 0.], [1.; 1.])];
ts = collect(0 : optparams.Δt : optparams.Δt * (optparams.nsteps - 1))
brstates = BoxRobots.BoxRobotState.(states)
brinputs = BoxRobots.BoxRobotInput.(inputs)
trajectory = BoxRobots.Trajectory(ts, BoxRobots.BoxRobotSimulationData.(ts, brstates, brinputs))
vis_options = BoxRobots.BoxRobotVisualizerOptions(force_arrow_normalizer=robot.m*norm(robot.g), playback_speed = 0.2)


# BoxRobots.draw_box_robot_state(vis, brstates[1], options = vis_options, input = brinputs[1])
;

In [9]:
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window(); sleep(1)
vis = Visualizer()
BoxRobots.playback_trajectory(vis, trajectory, options = vis_options)

In [2]:
function visualization_environment(;dist_to_left_wall=0.5, dist_to_right_wall=0.5)
    thickness = 0.1
    floor_poly = polyhedron_from_bounds([-1,1],[-thickness,0])
    floor = Surface(SimpleHRepresentation(floor_poly))

    right_wall_poly = polyhedron_from_bounds([dist_to_right_wall,
    dist_to_right_wall + thickness],[-0.1,1.5])
    right_wall = Surface(SimpleHRepresentation(right_wall_poly))

    left_wall_poly = polyhedron_from_bounds([-(dist_to_left_wall + thickness),
    -dist_to_left_wall],[-0.1,1.5])
    left_wall = Surface(SimpleHRepresentation(left_wall_poly))

    surfaces = [floor, right_wall, left_wall]
    Environment(surfaces)
end

visualization_environment (generic function with 1 method)

In [3]:
function trajopt_environment(robot::BoxRobotWithRotation2D; dist_to_left_wall=0.5, dist_to_right_wall=0.5)
    μ = 0.8
    maxforce = 2 * robot.m * norm(robot.g)
    environment = [contact_region([-dist_to_left_wall; 0.], [dist_to_right_wall; 0.], μ, maxforce);
        contact_region([-dist_to_left_wall; 0.], [-dist_to_left_wall; 1.], μ, maxforce);
        contact_region([dist_to_right_wall; 0.], [dist_to_right_wall; 1.], μ, maxforce);
        axis_aligned_free_box_region([-dist_to_left_wall; 0.], [dist_to_right_wall; 1.])]
end

trajopt_environment (generic function with 1 method)

In [4]:
method = :Logarithmic2D
disc_level = 2
optparams = MIQPTrajOptParams(verbose = false, nsteps = 10, disc_level = disc_level, bilinearmethod = method)
contact_point_descriptions = Dict(
    :foot => ContactPointDescription(2., SimpleVRepresentation([-0.5 -0.5; 0.5 -0.5; -0.5 -1.; 0.5 -1.])),
    :hand => ContactPointDescription(2., SimpleVRepresentation([-0.5 -0.5; 0.5 -0.5; -0.5 0.5; 0.5 0.5])))
robot = BoxRobotWithRotation2D(0., 1., 60., SVector(0, -9.81), contact_point_descriptions)
environment = trajopt_environment(robot)
contactpointstates = Dict(:foot => ContactPointState(SVector(0., 0.)), :hand => ContactPointState(SVector(0.7, 0.1)))
initialstate = BoxRobotWithRotation2DState(0., 0.1, SVector(0.0, 0.8), SVector(0.5, 0.), contactpointstates)
;

In [5]:
states, inputs, diagnostics = miqp_trajopt(robot, environment, initialstate, optparams);

LoadError: AssertionError: status == :Optimal

In [ ]:
ts = collect(0 : optparams.Δt : optparams.Δt * (optparams.nsteps - 1))
brstates = BoxRobots.BoxRobotState.(states)
brinputs = BoxRobots.BoxRobotInput.(inputs)
trajectory = BoxRobots.Trajectory(ts, BoxRobots.BoxRobotSimulationData.(ts, brstates, brinputs))
vis_options = BoxRobots.BoxRobotVisualizerOptions(force_arrow_normalizer=robot.m*norm(robot.g), playback_speed = 0.2)

In [16]:
# using MixedIntegerExperiments
# vis = MixedIntegerExperiments.plot_piecewise_mccormick(5, -1., 1., -1., 1.);

In [17]:
# using Base.Test
# # TODO: more checks on the final result

# zvals = similar(zs, Float64)
# for i = 1 : length(contacts), j = 1 : length(steps)
#     @test sum(getvalue(zs[contacts(i), steps(j)])) == 1
# end

In [18]:
# gr()
# plt = plot_environment(environment)
# bla = plot!(plt, [NaN], [NaN], lab = "CoM", markershape = :circle)
# xvals = getvalue.(rs[:x])
# zvals = getvalue.(rs[:z])
# anim = @animate for n = 1 : nsteps
#     plt[3] = ([xvals[n]], [zvals[n]])
# end

In [19]:
# gif(anim, loop = 3, fps = round(Int64, 1 / h))